# 遗传算法（Genetic Algorithm，GA）
是一种模拟自然选择和遗传机制的优化算法。
它是基于种群的迭代优化过程，常用于求解复杂的优化问题。

遗传算法步骤：
初始化种群：随机生成初始种群。
适应度评估：对每个个体进行适应度评估。
选择操作：根据适应度选择适应度高的个体进行繁殖。
交叉操作：选择两个个体并进行交叉，产生新的后代。
变异操作：对个体进行小范围的随机变异。
替换操作：根据适应度选择新一代种群。

# 示例：目标函数：f(x,y) = sin(x)·cos(y)+ x + y
# 约束：0 ≤ x ≤ 5, 0 ≤ y ≤ 5, x + y ≤ 5

In [1]:
import random
import numpy as np

# 目标函数
def objective(x, y):
    return np.sin(x) * np.cos(y) + x + y

# 检查约束是否满足
def is_feasible(x, y):
    return 0 <= x <= 5 and 0 <= y <= 5 and (x + y <= 5)

# 初始化种群（实数编码）
def initialize_population(size):
    population = []
    while len(population) < size:
        x = random.uniform(0, 5)
        y = random.uniform(0, 5)
        if is_feasible(x, y):
            population.append((x, y))
    return population

# 适应度函数（目标函数 + 罚函数）
def fitness(ind):
    x, y = ind
    if is_feasible(x, y):
        return objective(x, y)
    else:
        return -1e9  # 不满足约束的解赋予极低适应度

# 选择操作：轮盘赌选择
def selection(population):
    fitnesses = [fitness(ind) for ind in population]
    total_fit = sum(fitnesses)
    probs = [f / total_fit for f in fitnesses]
    selected = random.choices(population, weights=probs, k=len(population))
    return selected

# 交叉操作：模拟实数单点交叉
def crossover(p1, p2, rate=0.8):
    if random.random() < rate:
        alpha = random.random()
        c1 = (alpha * p1[0] + (1 - alpha) * p2[0],
              alpha * p1[1] + (1 - alpha) * p2[1])
        c2 = ((1 - alpha) * p1[0] + alpha * p2[0],
              (1 - alpha) * p1[1] + alpha * p2[1])
        return c1, c2
    return p1, p2

# 变异操作：微调 x, y
def mutate(ind, rate=0.1):
    x, y = ind
    if random.random() < rate:
        x += random.uniform(-0.2, 0.2)
    if random.random() < rate:
        y += random.uniform(-0.2, 0.2)
    # 保证边界
    x = max(0, min(5, x))
    y = max(0, min(5, y))
    return (x, y) if is_feasible(x, y) else ind

# 遗传算法主程序
def genetic_algorithm(pop_size=50, generations=100):
    population = initialize_population(pop_size)

    for gen in range(generations):
        population = selection(population)

        # 交叉与变异
        next_gen = []
        for i in range(0, len(population), 2):
            p1 = population[i]
            p2 = population[i+1] if i+1 < len(population) else p1
            c1, c2 = crossover(p1, p2)
            next_gen.append(mutate(c1))
            next_gen.append(mutate(c2))

        population = next_gen[:pop_size]

        # 输出当前最优解
        best = max(population, key=fitness)
        print(f"Gen {gen+1}: Best x={best[0]:.3f}, y={best[1]:.3f}, f={fitness(best):.4f}")

    best = max(population, key=fitness)
    print("\n✅ 最佳解：")
    print(f"x = {best[0]:.4f}, y = {best[1]:.4f}, f(x,y) = {fitness(best):.4f}")

# 运行算法
genetic_algorithm()

Gen 1: Best x=3.073, y=1.714, f=4.7778
Gen 2: Best x=3.073, y=1.714, f=4.7778
Gen 3: Best x=3.073, y=1.714, f=4.7778
Gen 4: Best x=3.008, y=1.767, f=4.7498
Gen 5: Best x=2.872, y=2.036, f=4.7882
Gen 6: Best x=2.719, y=2.057, f=4.5839
Gen 7: Best x=2.719, y=2.057, f=4.5839
Gen 8: Best x=2.385, y=1.935, f=4.0758
Gen 9: Best x=2.385, y=1.935, f=4.0758
Gen 10: Best x=2.322, y=1.940, f=3.9985
Gen 11: Best x=2.561, y=1.313, f=4.0134
Gen 12: Best x=2.282, y=1.389, f=3.8082
Gen 13: Best x=2.197, y=1.399, f=3.7349
Gen 14: Best x=2.071, y=1.821, f=3.6743
Gen 15: Best x=2.010, y=1.827, f=3.6080
Gen 16: Best x=2.001, y=1.693, f=3.5827
Gen 17: Best x=2.034, y=1.975, f=3.6568
Gen 18: Best x=1.968, y=1.859, f=3.5652
Gen 19: Best x=1.964, y=1.850, f=3.5594
Gen 20: Best x=1.966, y=1.884, f=3.5656
Gen 21: Best x=1.921, y=1.773, f=3.5051
Gen 22: Best x=1.972, y=1.942, f=3.5804
Gen 23: Best x=1.962, y=2.036, f=3.5831
Gen 24: Best x=1.884, y=2.189, f=3.5223
Gen 25: Best x=2.053, y=1.991, f=3.6830
Gen 26: B

# 集群智能优化寻找最优解
相当于在更复杂、群体协同搜索框架下，用遗传算法来寻找全局最优解。
这类问题往往属于高维非线性优化或者多峰优化问题。

## ✅ 示例问题：高维多峰函数优化（Rastrigin Function）

### 🎯 目标函数：

$$
f(x) = An + \sum_{i=1}^n \left[ x_i^2 - A \cdot \cos(2\pi x_i) \right], \quad A = 10
$$

---

- **定义域**：  \( x_i \in [-5.12, 5.12] \)
- **维度**：  通常设为 \( n = 10 \)
- **全局最优解**：

  $$
  f(0, 0, \ldots, 0) = 0
  $$

---

该函数具有大量局部极小值，是评估优化算法“全局搜索能力”的经典测试函数。


In [3]:
import random
import numpy as np
import plotly.graph_objs as go

# Rastrigin 函数定义
def rastrigin(x):
    A = 10
    n = len(x)
    return A * n + sum([xi**2 - A * np.cos(2 * np.pi * xi) for xi in x])

# 初始化种群（集群）
def initialize_population(pop_size, dim, bound):
    return [np.random.uniform(-bound, bound, dim).tolist() for _ in range(pop_size)]

# 适应度函数（最小化目标函数）
def fitness(ind):
    return -rastrigin(ind)  # GA 默认最大化，这里取负

# 轮盘赌选择
def selection(population):
    fitnesses = [fitness(ind) for ind in population]
    total_fit = sum(fitnesses)
    probs = [f / total_fit for f in fitnesses]
    selected = random.choices(population, weights=probs, k=len(population))
    return selected

# 实数交叉
def crossover(p1, p2, rate=0.9):
    if random.random() < rate:
        alpha = random.random()
        c1 = [(alpha * a + (1 - alpha) * b) for a, b in zip(p1, p2)]
        c2 = [(1 - alpha) * a + alpha * b for a, b in zip(p1, p2)]
        return c1, c2
    return p1[:], p2[:]

# 变异操作
def mutate(ind, rate=0.1, bound=5.12):
    return [np.clip(x + random.uniform(-0.2, 0.2), -bound, bound) if random.random() < rate else x
            for x in ind]

# 遗传算法主程序 + 可视化记录
def genetic_algorithm_cluster(pop_size=100, dim=2, generations=50, bound=5.12):
    population = initialize_population(pop_size, dim, bound)
    history = []
    best_history = []

    for gen in range(generations):
        population = selection(population)
        next_gen = []

        for i in range(0, len(population), 2):
            p1 = population[i]
            p2 = population[i + 1 if i + 1 < len(population) else 0]
            c1, c2 = crossover(p1, p2)
            next_gen.append(mutate(c1, bound=bound))
            next_gen.append(mutate(c2, bound=bound))

        population = next_gen[:pop_size]
        best = max(population, key=fitness)
        best_history.append(-fitness(best))
        history.append(np.array(population))
        print(f"Gen {gen+1}: Best fitness = {-fitness(best):.4f}")

    best = max(population, key=fitness)
    print("\n✅ 最佳解:")
    print(f"x = {np.round(best, 4)}")
    print(f"f(x) = {rastrigin(best):.6f}")

    plot_population_animation(history, bound)
    plot_convergence_curve(best_history)

# 可视化动画函数
def plot_population_animation(history, bound):
    frames = []
    for i, pop in enumerate(history):
        frames.append(go.Frame(
            data=[go.Scatter(x=pop[:, 0], y=pop[:, 1], mode='markers', marker=dict(size=6, color='blue'))],
            name=f"Gen {i+1}"
        ))

    fig = go.Figure(
        data=frames[0].data,
        layout=go.Layout(
            title="遗传算法种群进化动画",
            xaxis=dict(range=[-bound, bound], title='X'),
            yaxis=dict(range=[-bound, bound], title='Y'),
            updatemenus=[dict(type='buttons', showactive=False,
                              buttons=[dict(label='播放', method='animate', args=[None])])]
        ),
        frames=frames
    )
    fig.show()

# 收敛曲线图
def plot_convergence_curve(best_history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=best_history, mode='lines+markers', name='Best Fitness'))
    fig.update_layout(title='GA 收敛曲线', xaxis_title='Generation', yaxis_title='Best f(x)')
    fig.show()

# 运行
genetic_algorithm_cluster()

Gen 1: Best fitness = 5.6348
Gen 2: Best fitness = 2.0720
Gen 3: Best fitness = 5.6612
Gen 4: Best fitness = 1.7739
Gen 5: Best fitness = 2.9615
Gen 6: Best fitness = 4.3346
Gen 7: Best fitness = 1.4413
Gen 8: Best fitness = 1.5882
Gen 9: Best fitness = 2.6702
Gen 10: Best fitness = 4.8065
Gen 11: Best fitness = 5.3411
Gen 12: Best fitness = 4.7463
Gen 13: Best fitness = 6.2902
Gen 14: Best fitness = 18.6739
Gen 15: Best fitness = 21.8231
Gen 16: Best fitness = 24.1227
Gen 17: Best fitness = 26.0431
Gen 18: Best fitness = 27.7249
Gen 19: Best fitness = 30.3408
Gen 20: Best fitness = 31.9310
Gen 21: Best fitness = 28.4272
Gen 22: Best fitness = 30.0549
Gen 23: Best fitness = 31.9687
Gen 24: Best fitness = 35.8085
Gen 25: Best fitness = 31.2930
Gen 26: Best fitness = 35.1382
Gen 27: Best fitness = 34.9315
Gen 28: Best fitness = 33.9819
Gen 29: Best fitness = 37.1797
Gen 30: Best fitness = 35.4218
Gen 31: Best fitness = 31.0134
Gen 32: Best fitness = 36.0355
Gen 33: Best fitness = 32.1436